In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import time
import random 
from collections import Counter

In [67]:
import nltk
nltk.download("reuters") #using the reuters package

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [68]:
from nltk.corpus import reuters

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [70]:
np.random.seed(42)

In [71]:
print(len(reuters.fileids()))     # number of documents
print(reuters.categories()[:10])  # news categories


10788
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee']


In [72]:
# sentences = reuters.sents(categories=['trade', 'money-fx'])

sentences=reuters.sents()
sentences

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]

In [73]:
sentences=[[word.lower() for word in sentence] for sentence in sentences] # converting all words to lower case
sentences

[['asian',
  'exporters',
  'fear',
  'damage',
  'from',
  'u',
  '.',
  's',
  '.-',
  'japan',
  'rift',
  'mounting',
  'trade',
  'friction',
  'between',
  'the',
  'u',
  '.',
  's',
  '.',
  'and',
  'japan',
  'has',
  'raised',
  'fears',
  'among',
  'many',
  'of',
  'asia',
  "'",
  's',
  'exporting',
  'nations',
  'that',
  'the',
  'row',
  'could',
  'inflict',
  'far',
  '-',
  'reaching',
  'economic',
  'damage',
  ',',
  'businessmen',
  'and',
  'officials',
  'said',
  '.'],
 ['they',
  'told',
  'reuter',
  'correspondents',
  'in',
  'asian',
  'capitals',
  'a',
  'u',
  '.',
  's',
  '.',
  'move',
  'against',
  'japan',
  'might',
  'boost',
  'protectionist',
  'sentiment',
  'in',
  'the',
  'u',
  '.',
  's',
  '.',
  'and',
  'lead',
  'to',
  'curbs',
  'on',
  'american',
  'imports',
  'of',
  'their',
  'products',
  '.'],
 ['but',
  'some',
  'exporters',
  'said',
  'that',
  'while',
  'the',
  'conflict',
  'would',
  'hurt',
  'them',
  'in',


In [107]:
# corpus = sentences[:700] # taking small subset for faster training
corpus=sentences

In [75]:
flatten=lambda l:[word for sentence in l for word in sentence ] #flatten 1 level 
vocab=list(set(flatten(sentences)))

In [76]:
len(vocab)

31077

In [77]:
vocab.append('<UNK>')

In [78]:
voc_size=len(vocab)

In [79]:
vocab[:8]

['prove',
 'caesars',
 'piosec',
 'hun',
 'humidity',
 'inflammatory',
 'moura',
 'squire']

In [80]:
word2index={w:i for i,w in enumerate(vocab)}
print(word2index)

{'prove': 0, 'caesars': 1, 'piosec': 2, 'hun': 3, 'humidity': 4, 'inflammatory': 5, 'moura': 6, 'squire': 7, 'acquistions': 8, 'mdln': 9, 'baxter': 10, 'm': 11, 'committess': 12, 'dependant': 13, 'reprocess': 14, 'oranges': 15, 'melville': 16, 'leak': 17, 'ramar': 18, 'multimillions': 19, 'jwt': 20, 'fielder': 21, 'horse': 22, 'wae': 23, 'essxa': 24, 'cohen': 25, 'resumption': 26, 'xj': 27, 'stephan': 28, 'crossborder': 29, 'reader': 30, 'stepan': 31, 'scrip': 32, 'hayes': 33, 'wojnilower': 34, 'proves': 35, 'rib': 36, 'relying': 37, 'beijar': 38, 'expensive': 39, 'herbicides': 40, '503': 41, 'sorts': 42, 'catto': 43, 'competitor': 44, 'fw': 45, 'quix': 46, 'nl': 47, 'war': 48, 'preliminary': 49, 'cstk': 50, 'pate': 51, 'about': 52, 'tiny': 53, 'weakens': 54, 'mines': 55, 'denominated': 56, '304': 57, 'barlett': 58, 'awaits': 59, 'designing': 60, 'genecor': 61, '7239': 62, 'paranagua': 63, 'zy': 64, 'zimbabwe': 65, 'chrz': 66, 'sekiyu': 67, 'rigorous': 68, 'chester': 69, 'kalamazoo': 7

In [81]:
default_window_size=2

In [82]:
def random_batch(batch_size, word_sequence,window_size=2):
    
    # Make skip gram of one size window
    skip_grams = []
    # loop each word sequence
    # we starts from 1 because 0 has no context
    # we stop at second last for the same reason
    for sent in corpus:
        for index in range(window_size, len(sent) - window_size):       
            target = word2index[sent[index]]
            context = []        
            count = window_size # count of context words to pick on the left and right
            while count > 0:
                # for default window, it will get the left most and right most word
                # then the second left most and second right most word
                context.append(word2index[sent[index - count]])
                context.append(word2index[sent[index + count]])
                count -= 1
            for word in context:
                skip_grams.append([target, word])
    
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(skip_grams)), batch_size, replace=False) #randomly pick without replacement
        
    for index in random_index:
        random_inputs.append([skip_grams[index][0]])  # target, e.g., 2
        random_labels.append([skip_grams[index][1]])  # context word, e.g., 3
            
    return np.array(random_inputs), np.array(random_labels)

In [83]:
#testing the method
batch_size = 2 # mini-batch size
input_batch, target_batch = random_batch(batch_size, corpus)

print("Input: ", input_batch)
print("Target: ", target_batch)

Input:  [[17059]
 [22730]]
Target:  [[14488]
 [13893]]


In [84]:
class Skipgram(nn.Module):
    
    def __init__(self, vocab_size, emb_size):
        super(Skipgram,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, emb_size)
        self.embedding_u = nn.Embedding(vocab_size, emb_size)
    
    def forward(self, center_words, target_words, all_vocabs):
        center_embeds = self.embedding_v(center_words) # [batch_size, 1, emb_size]
        target_embeds = self.embedding_u(target_words) # [batch_size, 1, emb_size]
        all_embeds    = self.embedding_u(all_vocabs) #   [batch_size, voc_size, emb_size]
        
        scores      = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, 1, emb_size] @ [batch_size, emb_size, 1] = [batch_size, 1, 1] = [batch_size, 1]

        norm_scores = all_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, voc_size, emb_size] @ [batch_size, emb_size, 1] = [batch_size, voc_size, 1] = [batch_size, voc_size]

        nll = -torch.mean(torch.log(torch.exp(scores)/torch.sum(torch.exp(norm_scores), 1).unsqueeze(1))) # log-softmax
        # scalar (loss must be scalar)    
            
        return nll # negative log likelihood

In [85]:
batch_size     = 2 # mini-batch size
embedding_size = 2 #so we can later plot
model          = Skipgram(voc_size, embedding_size)
model_skipgram = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [86]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

#use for the normalized term in the probability calculation
all_vocabs = prepare_sequence(list(vocab), word2index).expand(batch_size, len(vocab))  # [batch_size, voc_size]
all_vocabs.shape

torch.Size([2, 31078])

In [87]:
def epoch_time(start_time, end_time):
    """ 
    Returns calculated time in minute and seconds 
    """
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [90]:
import time

# Training
num_epochs = 5000
for epoch in range(num_epochs):
    
    start = time.time()
    
    input_batch, target_batch = random_batch(batch_size, corpus)
    input_batch  = torch.LongTensor(input_batch)  #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch) #[batch_size, 1]

    #changing to cuda as device mode
    input_batch  = input_batch.to(device)
    target_batch = target_batch.to(device)
    all_vocabs   = all_vocabs.to(device)

    optimizer.zero_grad()
    loss = model_skipgram(input_batch, target_batch, all_vocabs)
    
    loss.backward()
    optimizer.step()
    
    end = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start, end)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")

Epoch: 1000 | cost: 9.811328 | time: 0m 6s
Epoch: 2000 | cost: 10.844407 | time: 0m 6s
Epoch: 3000 | cost: 9.638746 | time: 0m 5s
Epoch: 4000 | cost: 10.673711 | time: 0m 6s
Epoch: 5000 | cost: 10.095760 | time: 0m 6s


In [93]:
import os
save_dir = "models"
os.makedirs(save_dir, exist_ok=True)

In [97]:
torch.save(model_skipgram.state_dict(), 'models/skipgram_model.pth')

In [98]:
import pickle
pickle.dump(model_skipgram, open('models/skipgram.pkl', 'wb'))

In [99]:
word = vocab[0]

In [100]:
#numericalization
id = word2index[word]
id

0

2. With Negative Sampling

In [101]:
Z = 0.001

In [103]:
type(corpus)

list

In [ ]:
word_count = Counter(corpus)
word_count = Counter(flatten(corpus))
total_number_words = sum([c for w, c in word_count.items()])

TypeError: unhashable type: 'list'

In [ ]:
total_number_words

In [ ]:
unigram_table = []

for vo in vocab:
    uw = word_count[vo] / total_number_words
    uw_alpha = int((uw ** 0.75) / Z)
    unigram_table.extend([vo] * uw_alpha)

In [ ]:
Counter(unigram_table)

In [ ]:
def negative_sampling(targets, unigram_table, k):
    batch_size = targets.shape[0]
    neg_samples = []
    for i in range(batch_size):  #(1, k)
        target_index = targets[i].item()
        nsample      = []
        while (len(nsample) < k):
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))
        
    return torch.cat(neg_samples) #batch_size, k

In [ ]:
batch_size = 2
x, y = random_batch(batch_size, corpus)
x_tensor = torch.LongTensor(x)
y_tensor = torch.LongTensor(y)
x_tensor = x_tensor.to(device)
y_tensor = y_tensor.to(device)

In [ ]:
k = 5
neg_samples = negative_sampling(y_tensor, unigram_table, k)

In [ ]:
y_tensor[0]

In [ ]:
neg_samples[0]

In [ ]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
        self.logsigmoid        = nn.LogSigmoid()
    
    def forward(self, center, outside, negative):
        #center, outside:  (bs, 1)
        #negative       :  (bs, k)
        
        center_embed   = self.embedding_center(center) #(bs, 1, emb_size)
        outside_embed  = self.embedding_outside(outside) #(bs, 1, emb_size)
        negative_embed = self.embedding_outside(negative) #(bs, k, emb_size)

        
        uovc           = outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, 1)
        ukvc           = -negative_embed.bmm(center_embed.transpose(1, 2)).squeeze(2) #(bs, k)
        ukvc_sum       = torch.sum(ukvc, 1).reshape(-1, 1) #(bs, 1)
        
        loss           = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)
        
        return -torch.mean(loss)

In [ ]:
emb_size = 2
voc_size = len(vocab)
model = SkipgramNeg(voc_size, emb_size)
model_neg = model.to(device)
neg_samples = neg_samples.to(device)

In [ ]:
loss = model_neg(x_tensor, y_tensor, neg_samples)
loss

In [ ]:
optimizer = optim.Adam(model_neg.parameters(), lr=0.001)

In [ ]:
num_epochs = 5000

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, news_corpus)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)

    #move to cuda
    input_tensor = input_tensor.to(device)
    label_tensor = label_tensor.to(device)
    
    #predict
    neg_samples = negative_sampling(label_tensor, unigram_table, k)
    neg_samples = neg_samples.to(device)

    loss = model_neg(input_tensor, label_tensor, neg_samples)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    #print the loss
    if (epoch + 1) % 1000 == 0:
        print(f"Epoch {epoch+1:6.0f} | Loss: {loss:2.6f}")

In [ ]:
torch.save(model_neg.state_dict(), 'model/skipgram_neg_sample_model.pth')

In [ ]:
pickle.dump(model_neg, open('model/skipgram_neg_sample_model.pkl', 'wb'))

In [ ]:
def get_embed_neg_sample(word):
    id_tensor = torch.LongTensor([word2index[word]])
    id_tensor = id_tensor.to(device)
    v_embed = model_neg.embedding_center(id_tensor)
    u_embed = model_neg.embedding_outside(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return x, y

In [ ]:
government = get_embed_neg_sample('government')
officials = get_embed_neg_sample('officials')
administration = get_embed_neg_sample('administration')

In [ ]:
def get_skipgram(window_size = 2):
    # Make skip gram of custom size window
    skip_grams = []

    for sent in corpus:
        for target_index in range(window_size, len(sent) - window_size):
            target = sent[target_index]
            context = []
            count = window_size # count of context words to pick on the left and right
            while count > 0:
                # for default window, it will get the left most and right most word
                # then the second left most and second right most word
                context.append(sent[target_index - count])
                context.append(sent[target_index + count])
                count -= 1

            for word in context:
                skip_grams.append((target, word))
    return skip_grams

In [ ]:
skip_grams = get_skipgram(2)
skip_grams[:8]

In [ ]:
X_ik_skipgram = Counter(skip_grams)

In [ ]:
#simply a normalized function...don't worry too much
def weighting(w_i, w_j, X_ik):
        
    #check whether the co-occurrences exist between these two words
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        x_ij = 1  #if does not exist, set it to 1
                
    x_max = 100 #100 # fixed in paper  #cannot exceed 100 counts
    alpha = 0.75
    
    #if co-occurrence does not exceed 100, scale it based on some alpha
    if x_ij < x_max:
        result = (x_ij/x_max)**alpha  #scale it
    else:
        result = 1  #if is greater than max, set it to 1 maximum
    
    return result

In [ ]:
from itertools import combinations_with_replacement

X_ik = {}  #for keeping the co-occurences
weighting_dic = {} #scaling the percentage of sampling

for bigram in combinations_with_replacement(vocab, 2):
    if X_ik_skipgram.get(bigram) is not None:  #matches 
        co_occer = X_ik_skipgram[bigram]  #get the count from what we already counted
        X_ik[bigram] = co_occer + 1 # + 1 for stability issue
        X_ik[(bigram[1],bigram[0])] = co_occer+1   #count also for the opposite
    else:
        pass
        
    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

# print(f"{X_ik=}")
# print(f"{weighting_dic=}")

In [ ]:
def random_batch(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):
    
    #convert to id since our skip_grams is word, not yet id
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]
    
    random_inputs = []
    random_labels = []
    random_coocs  = []
    random_weightings = []
    random_index = np.random.choice(range(len(skip_grams_id)), batch_size, replace=False) #randomly pick without replacement
        
    for i in random_index:
        random_inputs.append([skip_grams_id[i][0]])  # target, e.g., 2
        random_labels.append([skip_grams_id[i][1]])  # context word, e.g., 3
        
        #get cooc
        pair = skip_grams[i]
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1
        random_coocs.append([math.log(cooc)])
        
        #get weighting
        weighting = weighting_dic[pair]
        random_weightings.append([weighting])
                    
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weightings)

In [ ]:
#testing the method
batch_size = 2 # mini-batch size
input_batch, target_batch, cooc_batch, weighting_batch = random_batch(batch_size, news_corpus, skip_grams, X_ik, weighting_dic)

print("Input: ", input_batch)
print("Target: ", target_batch)
print("Cooc: ", cooc_batch)
print("Weighting: ", weighting_batch)

In [ ]:
class GloVe(nn.Module):
    
    def __init__(self, vocab_size,embed_size):
        super(GloVe,self).__init__()
        self.embedding_center = nn.Embedding(vocab_size, embed_size) # center embedding
        self.embedding_outside = nn.Embedding(vocab_size, embed_size) # out embedding
        
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
    def forward(self, center_words, target_words, coocs, weighting):
        center_embeds = self.embedding_center(center_words) # [batch_size, 1, emb_size]
        target_embeds = self.embedding_outside(target_words) # [batch_size, 1, emb_size]
        
        center_bias = self.v_bias(center_words).squeeze(1)
        target_bias = self.u_bias(target_words).squeeze(1)
        
        inner_product = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, 1, emb_size] @ [batch_size, emb_size, 1] = [batch_size, 1, 1] = [batch_size, 1]
        
        #note that coocs already got log
        loss = weighting*torch.pow(inner_product +center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)

In [ ]:
batch_size     = 10 # mini-batch size
embedding_size = 2 #so we can later plot
model_glove    = GloVe(voc_size, embedding_size)
model_glove    = model_glove.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_glove.parameters(), lr=0.001)

In [ ]:
num_epochs = 5000
for epoch in range(num_epochs):
    
    start = time.time()
    
    input_batch, target_batch, cooc_batch, weighting_batch = random_batch(batch_size, news_corpus, skip_grams, X_ik, weighting_dic)
    input_batch  = torch.LongTensor(input_batch)         #[batch_size, 1]
    target_batch = torch.LongTensor(target_batch)        #[batch_size, 1]
    cooc_batch   = torch.FloatTensor(cooc_batch)         #[batch_size, 1]
    weighting_batch = torch.FloatTensor(weighting_batch) #[batch_size, 1]

    # to cuda
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    cooc_batch = cooc_batch.to(device)
    weighting_batch = weighting_batch.to(device)
    
    optimizer.zero_grad()
    loss = model_glove(input_batch, target_batch, cooc_batch, weighting_batch)
    
    loss.backward()
    optimizer.step()
    
    end = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start, end)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch: {epoch + 1} | cost: {loss:.6f} | time: {epoch_mins}m {epoch_secs}s")

In [ ]:
# save the model
torch.save(model_glove.state_dict(), 'model/glove_model.pth')

In [ ]:
# save the model using pickle
pickle.dump(model_glove, open('model/glove.pkl', 'wb'))

In [ ]:
def get_embed_glove(word):
    id_tensor = torch.LongTensor([word2index[word]])
    id_tensor = id_tensor.to(device)
    v_embed = model_glove.embedding_center(id_tensor)
    u_embed = model_glove.embedding_outside(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    x, y = word_embed[0][0].item(), word_embed[0][1].item()

    return x, y

In [ ]:
government = get_embed_glove('government')
officials = get_embed_glove('officials')
administration = get_embed_glove('administration')

In [ ]:
def cos_sim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

In [ ]:
print(f"government vs officials: {cos_sim(government, officials):.4f}")
print(f"government vs administration: {cos_sim(government, administration):.4f}")
print(f"government vs government: {cos_sim(government, government):.4f}")

Task 2

In [ ]:
def predict_word(word1, word2, word3, embeddings, word_to_index, index_to_word):
    # Get vectors for w1, w2, w3
    vec1 = np.array(embeddings(word1))
    vec2 = np.array(embeddings(word2))
    vec3 = np.array(embeddings(word3))

    # Calculate the predicted vector
    predicted_vec = vec1 - vec2 + vec3

    # Find the closest word by cosine similarity
    max_similarity = -1
    best_word = None
    for word, index in word_to_index.items():
        if word in [word1, word2, word3]:  # Skip the input words
            continue
        similarity = cos_sim(predicted_vec, embeddings(word))
        if similarity > max_similarity:
            max_similarity = similarity
            best_word = word

    return best_word

In [ ]:
# Evaluate accuracy
def evaluate_analogies(analogy_dataset, embeddings, word_to_index):
    correct = 0
    total = 0

    for analogy in analogy_dataset:
        word1, word2, word3, word4 = analogy
        if word1 not in word_to_index or word2 not in word_to_index or word3 not in word_to_index or word4 not in word_to_index:
            continue  # Skip if any word is not in the vocabulary
        predicted_word = predict_word(word1, word2, word3, embeddings, word_to_index, {v: k for k, v in word_to_index.items()})
        if predicted_word == word4:
            correct += 1
        total += 1

    return correct / total if total > 0 else 0

Dataset

In [ ]:
with open("capital-common-countries.txt", "r") as file:
    lines = file.readlines()

semantic_dataset = []
for line in lines:
    # Split the line into words
    words = line.strip().split()
    if len(words) == 4:
        semantic_dataset.append([words[0], words[1], words[2], words[3]])

In [ ]:
with open("past-tense.txt", "r") as file:
    lines = file.readlines()

past_tense_dataset = []
for line in lines:
    words = line.strip().split()
    if len(words) == 4:
        past_tense_dataset.append([words[0], words[1], words[2], words[3]])

In [ ]:
semantic_dataset[:5]

In [ ]:
past_tense_dataset[:5]